# Test de la reconstruction de mel spectrogram via MCNN et GRIFFIN LIM

In [ ]:
from final_model import AudioDataset, WAE
import matplotlib.pyplot as plt
import librosa as li
from util import MCNN, audio_utilities as au
import torch
from IPython.display import Audio

In [ ]:
MCNNNet = MCNN.build_pretrainedMCNN("new_MCNNtest16", "../pretrained/", 1, "cpu", 1, 500)[0]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MCNNNet = MCNNNet.to(device)
print(device)

In [ ]:
uku = AudioDataset("../notebooks/motu/motu_dataset/*.wav", process=False)
model = torch.load("../pretrained/model_20000_epoch.pt")
model.eval()
print("done!")

In [ ]:
plt.figure(figsize=(15,10))
plt.imshow(uku[6],aspect="auto", origin="lower", cmap="jet")
plt.show()

In [ ]:
idx = 9

In [ ]:
x = MCNNNet(uku[idx].unsqueeze(0).to(device)).squeeze(0).cpu().detach().numpy()
x = x[22050//5:22050]
au.save_audio_to_file(x/max(abs(x)), fs, "MCNN_ori.wav")

In [ ]:
Audio(x[22050//5:22050], rate=22050)

In [ ]:
x = MCNNNet(model(uku[idx].unsqueeze(0).to(device))).squeeze(0).cpu().detach().numpy()
x = x[22050//5:22050]
au.save_audio_to_file(x/max(abs(x)), fs, "MCNN_model.wav")

In [ ]:
Audio(x[22050//5:22050], rate=22050)

In [ ]:
x, fs = li.load(uku.liste[idx].replace(".pt",".wav"))
x = x[22050//5:22050]
au.save_audio_to_file(x/max(abs(x)), fs, "Ground_Truth.wav")

In [ ]:
Audio(x[22050//5:22050], rate=22050)

In [ ]:
mel = li.filters.mel(fs,2048,500)
STFT = mel.T.dot(uku[idx])

x = au.reconstruct_signal_griffin_lim(STFT.T, 2048, 256, 200)
x = x[22050//5:22050]
au.save_audio_to_file(x/max(abs(x)), fs, "Griffin_LIM_ori.wav")

In [ ]:
Audio(x,rate=fs)

In [ ]:
mel = li.filters.mel(fs,2048,500)
STFT = mel.T.dot(model(uku[idx].to(device).unsqueeze(0)).cpu().squeeze(0).detach().numpy())

x = au.reconstruct_signal_griffin_lim(STFT.T, 2048, 256, 200)
x = x[22050//5:22050]
au.save_audio_to_file(x/max(abs(x)), fs, "Griffin_LIM_model.wav")

In [ ]:
Audio(x,rate=fs)